In [1]:
from ultralytics import YOLO
import cv2
import math
import numpy as np
import cvzone


In [2]:
cap = cv2.VideoCapture('cars.mp4')

imported model

In [3]:
model = YOLO('./yolo-wts/yolov8n.pt')

import mask

In [4]:
mask = cv2.imread('mask.png')

In [5]:
from sort import *

In [6]:
tracker = Sort(max_age = 20, min_hits=3, iou_threshold=3)

In [7]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
image2 = cv2.imread('mask.png')
h2,w2,c2 = image2.shape
print(h2,w2,c2)
print(frame_height,frame_width)




720 1280 3
720 1280


In [8]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [9]:
#working one
mask = cv2.imread("mask.png")
 
# Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
 
limits = [400, 297, 673, 297]
totalCount = []
 
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)
 
    # imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    # img = cvzone.overlayPNG(img, imgGraphics, (0, 0))
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
 
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
 
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
 
    cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    # cv2.putText(img,str(len(totalCount)),(255,100),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    cv2.waitKey(1)



0: 384x640 4 cars, 231.4ms
Speed: 15.6ms preprocess, 231.4ms inference, 20.8ms postprocess per image at shape (1, 3, 384, 640)
[        572         321         651         429           4]
[        573         364         626         429           3]
[        378         348         497         460           2]
[        455         225         510         271           1]

0: 384x640 4 cars, 99.2ms
Speed: 15.7ms preprocess, 99.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[     568.66      320.48      651.34      435.52           4]
[     571.58      372.34      627.42      437.66           3]
[     373.49       349.5      494.51       465.5           2]
[     454.88       228.1      509.12       273.9           1]

0: 384x640 3 cars, 99.2ms
Speed: 0.0ms preprocess, 99.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[     555.36         379       631.4       475.3           3]
[      352.6      367.78      480.17      489.04           2]
[  

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


In [ ]:
# mask = cv2.imread('mask.png')
limits = [400,297,673,297]
detections = np.empty((0,5))
vehicles = ['car','motorbike','bus','truck']
while True:
    success, img  = cap.read()
    img_masked = cv2.bitwise_and(img,mask)
    results = model(img_masked,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            w,h = x2-x1, y2-y1
            #confidence
            conf = math.ceil(box.conf[0]*100)/100
            #class 
            cls = int(box.cls[0])
            classname = classNames[cls]
            if classname in vehicles and conf > 0.3:
                # cvzone.cornerRect(img,(x1,y1,x2,y2),l=9,rt=5)
                # cvzone.putTextRect(img,f'{classname} {conf}',(max(0,x1),max(35,y1)), (max(35,y1)),
                #                    thickness=3,offset=10)
                # cvzone.putTextRect(img, f'{classname} {conf}', (max(0, x1), max(35, y1)),
                                #    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currbox = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currbox))
            
        boxtracker = tracker.update(detections)
        #id will be provided by tracker method
        for results in boxtracker:
            x1,y1,x2,y2,id = map(int,results)
            # print(x1,y1,x2,y2,id)
            w,h=x2-x1,y2-y1
            cvzone.cornerRect(img,(x1,y1,x2,y2),l=9,rt=2,colorR=(255,0,255))    
            cvzone.putTextRect(img,f'{id}',(max(0,x1),max(35,y1)),scale=2,thickness=3,offset=10)
        # cvzone.cornerRect(img,(x1,y1,x2,y2),l=9,rt=5)
        cv2.imshow('image',img)
        cv2.waitKey(0)
        if cv2.getWindowProperty('image',cv2.WND_PROP_VISIBLE) < 1:
            break
            
            